In [ ]:
# | default_exp models

In [ ]:
# | export
import math

import torch
import torch.nn.functional as F
from torch import nn

from looped_experiments.nano_gpt import Block, LayerNorm

In [ ]:
# | export
class MaskedTransformer(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.transformer = nn.ModuleDict(dict(
            drop=nn.Dropout(cfg.dropout),
            h=nn.ModuleList([Block(cfg) for _ in range(cfg.n_layer)]),
        ))

    def forward(self, embs):
        x = self.transformer.drop(embs)
        for block in self.transformer.h: x = block(x)

        return x
    
def init_weights(module):
    if isinstance(module, nn.Linear):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        if module.bias is not None: torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

def init_all_params(model):
    # init all weights
    model.apply(init_weights)
    # apply special scaled init to the residual projections, per GPT-2 paper
    for pn, p in model.named_parameters():
        if pn.endswith('c_proj.weight'):
            torch.nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * model.cfg.n_layer))

In [ ]:
# | export
class Transformer(nn.Module):
    '''Transformer for tasks from in-context learning'''

    def __init__(self, cfg, freq=2):
        super().__init__()
        self.cfg = cfg
        self.freq = freq  # one input, one output, and so on
        self.backbone = MaskedTransformer(cfg)
        self.block_size = cfg.n_positions * freq + 1  # input, output pairs + 1 for the target
        self.read_in = nn.Linear(cfg.n_dims, cfg.n_embd)
        self.wpe=nn.Embedding(self.block_size, cfg.n_embd)
        
        self.ln_f=LayerNorm(cfg.n_embd, bias=cfg.bias)
        self.read_out = nn.Linear(cfg.n_embd, 1)
        init_all_params(self)

    def create_prompt(self, xs, ys):
        n_dims = xs.shape[-1]
        y_wide = F.pad(ys.unsqueeze(-1), (0, n_dims - 1), value=0)
        return torch.stack((xs, y_wide), dim=2).flatten(1, 2)
    
    def add_positional(self, embs):
        device = embs.device
        _, t, _ = embs.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device)
        return embs + self.wpe(pos)

    def forward(self, xs, ys):
        x = self.create_prompt(xs, ys)
        x = self.read_in(x)
        x = self.add_positional(x)
        x = self.backbone(x)
        x = self.ln_f(x)
        y = self.read_out(x).squeeze(-1)
        y = y[:, ::self.freq]  # only take the outputs (every other element)
        return y

In [ ]:
# | export
class LoopedTransformer(Transformer):
    '''Looped transformer for tasks from in-context learning'''

    def __init__(self, cfg):
        super().__init__(cfg, freq=2)
        self.n_loops, self.n_loop_window = cfg.n_loops, cfg.n_loop_window
        self.repeat_positional = cfg.repeat_positional
        self.repeat_ln = cfg.repeat_ln
        self.keep_n_tokens = cfg.keep_n_tokens
        self.token_dec = cfg.token_dec or 0

    def _model(self, z, x):
        z = z + x  # residual with addition
        if self.repeat_positional: z = self.add_positional(z)
        z = self.backbone(z)
        if self.repeat_ln: z = self.ln_f(z)
        return z

    def forward(self, xs, ys):
        horizon_start = max(0, self.n_loops - self.n_loop_window)
        x = self.create_prompt(xs, ys)
        b, t, _ = x.size()
        x = self.read_in(x)
        if not self.repeat_positional: x = self.add_positional(x)
        z = torch.zeros_like(x)
        keep = t
        preds = torch.zeros(self.n_loop_window, b, t // self.freq, device=x.device) + float('nan')
        for i in range(self.n_loops):
            if i < horizon_start:
                with torch.no_grad(): z = self._model(z, x)
                continue
            z = self._model(z, x)
            y = z if self.repeat_ln else self.ln_f(z)
            y = self.read_out(y).squeeze(-1)
            preds[i - horizon_start, :, -(x.size(1) // self.freq):] = y[:, (x.size(1) - 2) % self.freq::self.freq]
            # remove first tokens
            keep = max(self.keep_n_tokens or t, keep - self.token_dec)
            z = z[:, -keep:]
            x = x[:, -keep:]

        return preds


def loop_loss(preds, ys):
    return torch.nanmean((preds - ys.expand_as(preds)).square())

In [ ]:
# | export
def get_model(cfg):
    models = {
        'gpt2': Transformer,
        'gpt2_loop': LoopedTransformer
    }
    return models[cfg.family](cfg)

def get_loss(cfg):
    losses = {
        'gpt2': F.mse_loss,
        'gpt2_loop': loop_loss
    }
    return losses[cfg.family]

In [ ]:
from looped_experiments.tasks import LinearRegression
from looped_experiments.utils import get_config, show_config

In [ ]:
cfg = get_config(overrides=['model=loop', 'model.token_dec=2', 'model.keep_n_tokens=5'])
# show_config(cfg)

In [ ]:
model = LoopedTransformer(cfg.model).to('cuda')

In [ ]:
xs,ys = LinearRegression(64,**cfg.task)()
xs, ys = xs.to('cuda'), ys.to('cuda')

In [ ]:
res = model(xs, ys)

In [ ]:
loop_loss(res, ys)

tensor(9.5480, device='cuda:0', grad_fn=<DivBackward0>)